In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import plot_importance


In [3]:
#read data

In [4]:
df = pd.read_csv('./CICIDS_2018/IDS.csv',index_col=[0])


In [5]:
df

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
5631664,2116.0,1.0,1.0,44.0,82.0,44.0,44.0,44.0,0.0,82.0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3099750,1057.0,1.0,1.0,41.0,103.0,41.0,41.0,41.0,0.0,103.0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1825967,66555644.0,35.0,46.0,3886.0,5154.0,583.0,0.0,111.028571,174.330735,1430.0,...,20.0,8191292.0,0.0,8191292.0,8191292.0,58302685.0,0.0,58302685.0,58302685.0,Benign
7113878,2462909.0,8.0,8.0,1032.0,1466.0,565.0,0.0,129.0,190.919579,1149.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1566689,322.0,1.0,1.0,47.0,63.0,47.0,47.0,47.0,0.0,63.0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420640,18184,1,1,31,87,31,31,31,0,87,...,8,0,0,0,0,0,0,0,0,Infilteration
490836,14208808,11,8,674,5918,517,0,61.27272727,155.8294522,1460,...,20,3051885,0,3051885,3051885,6014723,0,6014723,6014723,Infilteration
537625,5124568,3,1,0,0,0,0,0,0,0,...,20,0,0,0,0,0,0,0,0,Infilteration
21838,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label


In [6]:
df.drop(df.loc[df['Label'] == 'Label'].index, inplace=True)

In [7]:
df.Label.value_counts()

Benign                      674154
DDOS attack-HOIC             34301
DDoS attacks-LOIC-HTTP       28810
DoS attacks-Hulk             23096
Bot                          14116
FTP-BruteForce                9668
SSH-Bruteforce                9379
Infilteration                 8096
DoS attacks-SlowHTTPTest      6994
DoS attacks-GoldenEye         2075
DoS attacks-Slowloris          550
DDOS attack-LOIC-UDP            86
Brute Force -Web                30
Brute Force -XSS                12
SQL Injection                    5
Name: Label, dtype: int64

In [8]:
df.keys()

Index(['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s',
       'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
   

In [9]:
len(df.keys())

77

### Preprocessing (normalization and padding values)

In [10]:
labelencoder = LabelEncoder()
df.iloc[:, -1] = labelencoder.fit_transform(df.iloc[:, -1])

In [11]:
df = df.astype(float)

In [12]:
# Z-score normalization
features = df.dtypes.index
df[features] = df[features].apply(
    lambda x: (x - x.mean()) / (x.std()))
print(df[features])
# Fill empty values by 0
df = df.fillna(0)


         Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  TotLen Fwd Pkts  \
5631664      -0.394745     -0.015187     -0.035153        -0.018825   
3099750      -0.394780     -0.015187     -0.035153        -0.018885   
1825967       1.759464      0.006932      0.269120         0.057650   
7113878      -0.315094     -0.010633      0.012178         0.000841   
1566689      -0.394803     -0.015187     -0.035153        -0.018765   
...                ...           ...           ...              ...   
469282       -0.394794     -0.015187     -0.035153        -0.019064   
504000       -0.349719     -0.010633      0.005416         0.003150   
420640       -0.394225     -0.015187     -0.035153        -0.019084   
490836        0.065098     -0.008681      0.012178        -0.006285   
537625       -0.228941     -0.013886     -0.035153        -0.019701   

         TotLen Bwd Pkts  Fwd Pkt Len Max  Fwd Pkt Len Min  Fwd Pkt Len Mean  \
5631664        -0.021068        -0.508047         1.367593         

### Data sampling

In [13]:
labelencoder = LabelEncoder()
df.iloc[:, -1] = labelencoder.fit_transform(df.iloc[:, -1])

In [14]:
df.Label.value_counts()

0     674154
4      34301
6      28810
8      23096
1      14116
11      9668
14      9379
12      8096
9       6994
7       2075
10       550
5         86
2         30
3         12
13         5
Name: Label, dtype: int64

In [15]:
df = df[~(df.isin([np.inf, -np.inf]).any(axis=1))]

In [16]:
# retain the minority class instances
df_minor = df[(df['Label'] == 0) | (df['Label'] == 4) | (df['Label'] == 6) | (df['Label'] == 8) | (
    df['Label'] == 1) | (df['Label'] == 11) | (df['Label'] == 14) | (df['Label'] == 12) | (df['Label'] == 9) | (df['Label'] == 7) ]
df_major = df.drop(df_minor.index)


In [17]:
X = df_major.drop(['Label'],axis=1)
X = X[X.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)]
X = X.astype(np.float64, copy=False)

In [18]:
y = df_major.iloc[:, -1].values.reshape(-1, 1)
y = np.ravel(y)
print(y)

[10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10

In [19]:
X[X > 1e308] = 0

In [20]:
print(np.any(np.isinf(X)))

False


In [21]:
# use k-means to cluster the data samples and select a proportion of data from each cluster
from sklearn.cluster import KMeans
# from default to the number of labels
range_n_clusters = [4,5,6,7,8,9]
silhouette_avg = []
for num_clusters in range_n_clusters:
    # initialise kmeans 
    kmeans = KMeans(init='k-means++', n_clusters=num_clusters)
    kmeans.fit(X)
    cluster_labels = kmeans.labels_
    # Compute the silhouette scores for each sample
    silhouette_avg.append(silhouette_score(X, cluster_labels))
    
for i in range(len(silhouette_avg)):
    print(
        "for n_clusters : ",
        range_n_clusters[i],
        ", the average silhouette_score is :",
        silhouette_avg[i]
    )
#best score is while n_clusters=8

for n_clusters :  4 , the average silhouette_score is : 0.7857904174523094
for n_clusters :  5 , the average silhouette_score is : 0.7566007580793052
for n_clusters :  6 , the average silhouette_score is : 0.7581060920365768
for n_clusters :  7 , the average silhouette_score is : 0.7828588390658231
for n_clusters :  8 , the average silhouette_score is : 0.7811733583293067
for n_clusters :  9 , the average silhouette_score is : 0.7341825501731775


In [22]:
kmeans = KMeans(init='k-means++', n_clusters=8)
kmeans.fit_predict(X)

array([0, 0, 0, 0, 3, 2, 2, 0, 2, 0, 2, 2, 0, 2, 3, 2, 0, 2, 0, 3, 0, 2,
       2, 2, 2, 3, 0, 0, 3, 3, 3, 3, 2, 0, 0, 3, 0, 2, 2, 3, 2, 2, 3, 3,
       2, 2, 0, 0, 2, 0, 0, 0, 3, 0, 2, 0, 0, 3, 3, 0, 2, 0, 2, 0, 0, 0,
       3, 3, 3, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 2, 2, 0, 3, 2, 2, 2,
       3, 2, 3, 0, 0, 2, 2, 0, 2, 2, 0, 2, 0, 3, 2, 2, 2, 2, 3, 0, 2, 3,
       0, 2, 2, 3, 0, 0, 2, 0, 0, 0, 2, 2, 3, 3, 2, 0, 2, 3, 0, 0, 3, 2,
       0, 2, 2, 2, 0, 0, 0, 2, 3, 2, 2, 0, 2, 3, 3, 2, 0, 3, 2, 0, 0, 0,
       2, 0, 2, 2, 2, 0, 2, 2, 0, 2, 0, 3, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0,
       2, 0, 2, 2, 2, 3, 0, 0, 0, 0, 0, 0, 0, 3, 2, 3, 0, 2, 0, 2, 2, 3,
       2, 0, 3, 3, 0, 3, 3, 0, 3, 3, 0, 0, 3, 0, 0, 2, 2, 2, 2, 3, 2, 2,
       3, 0, 0, 3, 0, 0, 0, 2, 0, 2, 0, 3, 3, 2, 2, 0, 2, 2, 0, 0, 0, 3,
       0, 3, 3, 3, 2, 3, 0, 2, 0, 3, 2, 0, 2, 2, 2, 2, 0, 2, 3, 0, 0, 2,
       2, 2, 3, 0, 2, 2, 2, 0, 2, 3, 2, 3, 3, 3, 2, 2, 0, 0, 0, 2, 3, 2,
       2, 2, 2, 3, 2, 2, 2, 2, 3, 3, 0, 2, 0, 2, 2,

In [23]:
klabel = kmeans.labels_
df_major['klabel'] = klabel

In [24]:
df_major['klabel'].value_counts()

0    169
2    156
3     79
1     37
4      9
5      6
6      6
7      3
Name: klabel, dtype: int64

In [25]:
cols = list(df_major)
cols.insert(78, cols.pop(cols.index('Label')))
df_major = df_major.loc[:, cols]

In [26]:
df_major

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,klabel,Label
45582,-0.297083,-0.013886,-0.041915,-0.019701,-0.021452,-0.650602,-0.459452,-0.807009,-0.601549,-0.704871,...,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,0,10
45980,-0.296937,-0.013886,-0.041915,-0.019701,-0.021452,-0.650602,-0.459452,-0.807009,-0.601549,-0.704871,...,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,0,10
43613,-0.296816,-0.013886,-0.041915,-0.019701,-0.021452,-0.650602,-0.459452,-0.807009,-0.601549,-0.704871,...,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,0,10
35702,-0.296300,-0.013886,-0.041915,-0.019701,-0.021452,-0.650602,-0.459452,-0.807009,-0.601549,-0.704871,...,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,0,10
30816,2.841982,-0.014536,-0.028392,-0.019382,-0.021452,-0.624683,-0.127262,-0.678681,-0.601549,-0.704871,...,-0.069432,-0.057395,-0.079482,-0.054961,5.994517,-0.090334,5.894466,6.049624,3,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,-0.232787,-0.012585,-0.021630,-0.006842,-0.019750,1.442370,-0.459452,1.265483,1.851041,0.027772,...,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,0,2
1229,-0.394795,-0.014536,-0.041915,-0.019701,-0.021452,-0.650602,-0.459452,-0.807009,-0.601549,-0.704871,...,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,0,3
1075,-0.394809,-0.014536,-0.041915,-0.019701,-0.021452,-0.650602,-0.459452,-0.807009,-0.601549,-0.704871,...,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,0,3
1189,-0.394810,-0.014536,-0.041915,-0.019701,-0.021452,-0.650602,-0.459452,-0.807009,-0.601549,-0.704871,...,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,0,3


In [27]:
def typicalSampling(group):
    name = group.name
    frac = 100
    return group.sample(frac=frac, replace= True)

result = df_major.groupby('klabel', group_keys=False
).apply(typicalSampling)


In [28]:
result['Label'].value_counts()

10    37672
5      5500
2      2598
3       618
13      112
Name: Label, dtype: int64

In [29]:
result = result.drop(['klabel'], axis=1)
result = result.append(df_minor)

In [30]:
result

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
50387,-0.296443,-0.013886,-0.041915,-0.019701,-0.021452,-0.650602,-0.459452,-0.807009,-0.601549,-0.704871,...,2.859529,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,10
39981,-0.296171,-0.013886,-0.041915,-0.019701,-0.021452,-0.650602,-0.459452,-0.807009,-0.601549,-0.704871,...,2.859529,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,10
32972,-0.394804,-0.015187,-0.028392,-0.014963,-0.019039,0.120493,9.423202,3.010740,-0.601549,0.333711,...,1.820173,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,10
49077,-0.296164,-0.013886,-0.041915,-0.019701,-0.021452,-0.650602,-0.459452,-0.807009,-0.601549,-0.704871,...,2.859529,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,10
357,3.251156,-0.013886,-0.041915,-0.019701,-0.021452,-0.650602,-0.459452,-0.807009,-0.601549,-0.704871,...,-2.337249,-0.069433,-0.057395,-0.079483,-0.054963,3.244164,-0.090311,3.186388,3.278090,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469282,-0.394794,-0.015187,-0.035153,-0.019064,-0.021227,-0.546926,0.869308,-0.293698,-0.601549,-0.608258,...,-1.297894,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,12
504000,-0.349719,-0.010633,0.005416,0.003150,-0.014058,1.542807,-0.459452,1.494869,1.335137,1.656092,...,0.261140,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,12
420640,-0.394225,-0.015187,-0.035153,-0.019084,-0.021045,-0.550165,0.827784,-0.309739,-0.601549,-0.529761,...,-1.297894,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,12
490836,0.065098,-0.008681,0.012178,-0.006285,0.006223,1.024423,-0.459452,0.175864,0.721351,2.233753,...,0.261140,1.127449,-0.057395,0.827703,1.360008,0.076554,-0.090334,0.067462,0.086090,12


In [31]:
def sample(group):
    frac = 0.09
    return group.sample(frac=frac, replace=True)


sample_df = result.groupby('Label', group_keys=False).apply(sample)


In [32]:
sample_df

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
6404770,0.585900,-0.008681,0.032463,0.011490,0.000106,3.402506,-0.459452,1.478099,2.566517,2.233753,...,0.261140,-0.041288,0.010712,-0.022290,-0.049706,0.328468,-0.086003,0.315767,0.339418,0
5129635,-0.394813,-0.015187,-0.035153,-0.019701,-0.021452,-0.650602,-0.459452,-0.807009,-0.601549,-0.704871,...,0.261140,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,0
6235035,-0.389545,-0.013886,-0.035153,-0.019701,-0.021452,-0.650602,-0.459452,-0.807009,-0.601549,-0.704871,...,0.261140,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,0
4930595,-0.394809,-0.013886,-0.041915,-0.019084,-0.021452,-0.550165,-0.459452,-0.641252,-0.449607,-0.704871,...,0.261140,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,0
299427,3.421725,-0.005428,0.045986,0.000125,-0.002999,0.483361,-0.459452,0.191541,0.242479,2.173370,...,0.261140,0.096259,0.144986,0.111360,0.039139,3.377080,0.312963,3.346585,3.382018,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270611,-0.384595,-0.001525,0.106841,0.018994,-0.008989,1.422931,-0.459452,0.610429,0.568877,1.259579,...,1.820173,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,14
337970,-0.394814,-0.015187,-0.035153,-0.019701,-0.021452,-0.650602,-0.459452,-0.807009,-0.601549,-0.704871,...,1.820173,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,14
320610,-0.394814,-0.015187,-0.035153,-0.019701,-0.021452,-0.650602,-0.459452,-0.807009,-0.601549,-0.704871,...,1.820173,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,14
322104,-0.382990,-0.002176,0.106841,0.018676,-0.008989,1.422931,-0.459452,0.665704,0.584855,1.259579,...,1.820173,-0.069433,-0.057395,-0.079483,-0.054963,-0.302176,-0.090334,-0.305447,-0.295557,14


In [33]:
sample_df.to_csv('./CICIDS_2018/CICIDS2018_cleaned.csv', index=0)

## Read the cleaned CICIDS2018 dataset
Due to the large size of this dataset, the sampled subsets of CICIDS2018 is used. The subsets are in the "CICIDS_2018" folder.

In [34]:
#Read dataset
df = pd.read_csv('./CICIDS_2018/CICIDS2018_cleaned.csv')

In [35]:
df.keys()

Index(['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s',
       'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
   

In [36]:
df.Label.value_counts()

0     60674
10     3390
4      3087
6      2593
8      2079
1      1270
11      870
14      844
12      729
9       629
5       495
2       234
7       187
3        56
13       10
Name: Label, dtype: int64

In [37]:
df[df > 1e308] = 0

In [38]:
X = df.drop(['Label'], axis=1).values
y = df.iloc[:, -1].values.reshape(-1, 1)
y = np.ravel(y)


In [39]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    train_size=0.7,
    test_size=0.3,
    random_state=0,
    stratify=y
)

## Feature engineering
### Feature selection by information gain

In [40]:
from sklearn.feature_selection import mutual_info_classif
importances = mutual_info_classif(X_train, y_train)

In [41]:
print(importances)

[5.79967614e-01 2.66691682e-01 2.65798264e-01 4.72653877e-01
 3.40347465e-01 4.59568835e-01 1.38610981e-01 4.55225701e-01
 3.01630073e-01 3.36519207e-01 7.24348189e-02 3.43978571e-01
 2.51883992e-01 0.00000000e+00 0.00000000e+00 5.63024991e-01
 3.59914073e-01 5.84242730e-01 3.99719146e-01 5.42884777e-01
 5.32619380e-01 3.04035048e-01 5.44625204e-01 4.35434955e-01
 2.71470018e-01 2.61339189e-01 2.21318534e-01 2.70875629e-01
 2.33912796e-01 2.87995161e-02 8.58382972e-04 0.00000000e+00
 0.00000000e+00 5.30491982e-01 3.84157826e-01 5.58920986e-01
 4.36992236e-01 9.64284990e-02 4.59370628e-01 4.50418948e-01
 4.24288171e-01 4.24268292e-01 4.56444479e-03 3.09636572e-02
 4.23781197e-02 4.91333029e-02 6.26303327e-02 1.91706061e-02
 1.70561581e-04 4.36696891e-02 4.55806303e-02 4.49480782e-01
 4.52736098e-01 3.40346185e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.03354974e-03 9.49608405e-04
 2.69377255e-01 4.69750304e-01 2.61190831e-01 3.39600081e-01
 6.85285317e-01 3.382298

In [42]:
# calculate the sum of importance scores
f_list = sorted(
    zip(map(lambda x: round(x, 4), importances),
    features), reverse=True)
Sum = 0
fs = []
for i in range(0, len(f_list)):
    Sum = Sum + f_list[i][0]
    fs.append(f_list[i][1])

In [43]:
# select the important features from top to bottom until the accumulated importance reaches 95%
f_list2 = sorted(
    zip(map(lambda x: round(x, 4), importances/Sum), features), reverse=True)
Sum2 = 0
fs = []
for i in range(0, len(f_list2)):
    Sum2 = Sum2 + f_list2[i][0]
    fs.append(f_list2[i][1])
    if Sum2 >= 0.95:
        break

In [44]:
X_fs = df[fs].values

In [45]:
print(X_fs)

[[-0.03566032  0.22436237  0.58590017 ...  0.32846848 -0.45945226
  -0.04128834]
 [-0.52968658 -0.35884169 -0.39481337 ... -0.30217581 -0.45945226
  -0.06943273]
 [-0.03566032 -0.34938003 -0.38954514 ... -0.30217581 -0.45945226
  -0.06943273]
 ...
 [-0.52635399 -0.3588421  -0.3948136  ... -0.30217581 -0.45945226
  -0.06943273]
 [ 1.11784934 -0.3511747  -0.38299045 ... -0.30217581 -0.45945226
  -0.06943273]
 [ 1.11784934 -0.35281796 -0.38324211 ... -0.30217581 -0.45945226
  -0.06943273]]


In [46]:
X_fs.shape

(77147, 49)

### Feature selection by Fast Correlation Based Filter (FCBF)

In [47]:
from FCBF_module import FCBF, FCBFK, FCBFiP, get_i
fcbf = FCBFK(k=21)

In [48]:
X_fss = fcbf.fit_transform(X_fs, y)

In [49]:
X_fss.shape

(77147, 21)

### Re-split train & test sets after feature selection

In [50]:
X_train, X_test, y_train, y_test = train_test_split(
    X_fss, y, stratify=y, train_size=0.7, test_size=0.3, random_state=0)

In [51]:
X_train.shape

(54002, 21)

In [52]:
pd.Series(y_train).value_counts()

0     42471
10     2373
4      2161
6      1815
8      1455
1       889
11      609
14      591
12      510
9       440
5       347
2       164
7       131
3        39
13        7
dtype: int64

### SMOTE to solve class-imbalance

In [53]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(n_jobs=-1, sampling_strategy={
    1:889*3,
    11:609*3,
    14:591*7,
    12:510*12,
    9:440*14,
    5:347*11,
    2:161*17,
    7:131*17,
    3:38*70,
    13:6*110
})


In [54]:
X_train, y_train = smote.fit_resample(X_train, y_train)

In [55]:
pd.Series(y_train).value_counts()

0     42471
9      6160
12     6120
14     4137
5      3817
2      2737
1      2667
3      2660
10     2373
7      2227
4      2161
11     1827
6      1815
8      1455
13      660
dtype: int64

## Machine learning model training
### Training decision tree, random forest, extra trees, XGBoost

#### Apply decision tree

In [56]:
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_train, y_train)
dt_score = dt.score(X_test, y_test)
y_predict = dt.predict(X_test)
y_true = y_test
print('Accuracy of DT: ' + str(dt_score))
precision, recall, fscore, none = precision_recall_fscore_support(
    y_true, y_predict, average='weighted')
print('Precision of DT: '+(str(precision)))
print('Recall of DT: '+(str(recall)))
print('F1-score of DT: '+(str(fscore)))
print(classification_report(y_true, y_predict))
cm = confusion_matrix(y_true, y_predict)

Accuracy of DT: 0.9684597105206308
Precision of DT: 0.9782721298063981
Recall of DT: 0.9684597105206308
F1-score of DT: 0.9718564665939484
              precision    recall  f1-score   support

           0       0.99      0.98      0.98     18203
           1       0.98      0.99      0.99       381
           2       0.97      1.00      0.99        70
           3       1.00      1.00      1.00        17
           4       1.00      1.00      1.00       926
           5       0.99      1.00      1.00       148
           6       0.99      1.00      1.00       778
           7       0.98      0.95      0.96        56
           8       0.99      1.00      1.00       624
           9       0.56      0.96      0.71       189
          10       0.99      1.00      1.00      1017
          11       0.95      0.47      0.63       261
          12       0.10      0.20      0.14       219
          13       0.75      1.00      0.86         3
          14       1.00      1.00      1.00       

#### Hyperparameter optimization (HPO) of decision tree using Bayesian optimization with tree-based Parzen estimator (BO-TPE)

In [67]:
# Hyperparameter optimization of decision tree
from hyperopt import hp, fmin, tpe, STATUS_OK
# Define the objective function
def objective(params):
    params = {
        "min_samples_leaf": int(params['min_samples_leaf']),
        'max_depth': int(params['max_depth']),
        "min_samples_split": int(params['min_samples_split']),
        'max_features': int(params['max_features']),
        "criterion": str(params['criterion'])
    }
    clf = DecisionTreeClassifier(**params)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)

    return {'loss': -score, 'status': STATUS_OK}


# Define the hyperparameter configuration space
space = {
    "min_samples_leaf": hp.quniform('min_samples_leaf', 1, 11, 1),
    'max_depth': hp.quniform('max_depth', 5, 50, 1),
    "min_samples_split": hp.quniform('min_samples_split', 2, 11, 1),
    "max_features": hp.quniform('max_features', 1, 21, 1),
    "criterion": hp.choice('criterion', ['gini', 'entropy'])
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=77)
print("Decision tree: Hyperopt estimated optimum {}".format(best))


100%|██████████| 77/77 [00:34<00:00,  2.24trial/s, best loss: -0.9719161806005617]
Decision tree: Hyperopt estimated optimum {'criterion': 1, 'max_depth': 25.0, 'max_features': 6.0, 'min_samples_leaf': 1.0, 'min_samples_split': 8.0}


In [62]:
dt_hpo = DecisionTreeClassifier(
    min_samples_leaf=1, max_depth=16, min_samples_split=11, max_features=20, criterion='entropy')
dt_hpo.fit(X_train, y_train)
dt_score = dt_hpo.score(X_test, y_test)
y_predict = dt_hpo.predict(X_test)
y_true = y_test
print('Accuracy of DT: ' + str(dt_score))
precision, recall, fscore, none = precision_recall_fscore_support(
    y_true, y_predict, average='weighted')
print('Precision of DT: '+(str(precision)))
print('Recall of DT: '+(str(recall)))
print('F1-score of DT: '+(str(fscore)))
print(classification_report(y_true, y_predict))
cm = confusion_matrix(y_true, y_predict)


Accuracy of DT: 0.9720025923525599
Precision of DT: 0.9782357336719879
Recall of DT: 0.9720025923525599
F1-score of DT: 0.973721467384721
              precision    recall  f1-score   support

           0       0.99      0.98      0.99     18203
           1       0.98      0.99      0.98       381
           2       0.96      1.00      0.98        70
           3       1.00      1.00      1.00        17
           4       1.00      1.00      1.00       926
           5       0.99      1.00      1.00       148
           6       1.00      1.00      1.00       778
           7       0.96      0.89      0.93        56
           8       1.00      1.00      1.00       624
           9       0.56      0.96      0.71       189
          10       1.00      1.00      1.00      1017
          11       0.94      0.46      0.62       261
          12       0.12      0.19      0.15       219
          13       1.00      1.00      1.00         3
          14       1.00      1.00      1.00       2

In [63]:
dt_train = dt_hpo.predict(X_train)
dt_test = dt_hpo.predict(X_test)

### Apply RF

In [60]:
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
rf_score = rf.score(X_test, y_test)
y_predict = rf.predict(X_test)
y_true = y_test
print('Accuracy of RF: ' + str(rf_score))
precision, recall, fscore, none = precision_recall_fscore_support(
    y_true, y_predict, average='weighted')
print('Precision of RF: '+(str(precision)))
print('Recall of RF: '+(str(recall)))
print('F1-score of RF: '+(str(fscore)))
print(classification_report(y_true, y_predict))
cm = confusion_matrix(y_true, y_predict)

Accuracy of RF: 0.9766256210844675
Precision of RF: 0.9792517959112764
Recall of RF: 0.9766256210844675
F1-score of RF: 0.9759718240312625
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     18203
           1       0.98      0.99      0.98       381
           2       1.00      1.00      1.00        70
           3       0.94      1.00      0.97        17
           4       0.99      1.00      1.00       926
           5       0.99      1.00      1.00       148
           6       0.99      1.00      1.00       778
           7       0.98      0.96      0.97        56
           8       1.00      1.00      1.00       624
           9       0.54      0.99      0.70       189
          10       1.00      1.00      1.00      1017
          11       0.99      0.40      0.57       261
          12       0.20      0.19      0.19       219
          13       1.00      1.00      1.00         3
          14       1.00      1.00      1.00       

#### Hyperparameter optimization (HPO) of random forest using Bayesian optimization with tree-based Parzen estimator (BO-TPE)

In [68]:
# Hyperparameter optimization of random forest
from hyperopt import hp, fmin, tpe, STATUS_OK
# Define the objective function

def objective(params):
    params = {
        'n_estimators': int(params['n_estimators']),
        'max_depth': int(params['max_depth']),
        'max_features': int(params['max_features']),
        "min_samples_split": int(params['min_samples_split']),
        "min_samples_leaf": int(params['min_samples_leaf']),
        "criterion": str(params['criterion'])
    }
    clf = RandomForestClassifier(**params)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)

    return {'loss': -score, 'status': STATUS_OK}


# Define the hyperparameter configuration space
space = {
    'n_estimators': hp.quniform('n_estimators', 10, 200, 1),
    'max_depth': hp.quniform('max_depth', 5, 50, 1),
    "max_features": hp.quniform('max_features', 1, 21, 1),
    "min_samples_split": hp.quniform('min_samples_split', 2, 11, 1),
    "min_samples_leaf": hp.quniform('min_samples_leaf', 2, 11, 1),
    "criterion": hp.choice('criterion', ['gini', 'entropy'])
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=22)
print("Random Forest: Hyperopt estimated optimum {}".format(best))

100%|██████████| 21/21 [11:46<00:00, 33.65s/trial, best loss: -0.9791747677684165]
Random Forest: Hyperopt estimated optimum {'criterion': 0, 'max_depth': 37.0, 'max_features': 14.0, 'min_samples_leaf': 4.0, 'min_samples_split': 5.0, 'n_estimators': 107.0}


In [69]:
rf_hpo = RandomForestClassifier(n_estimators=188,              
                                min_samples_leaf=2,
                                max_depth=15, min_samples_split=4, max_features=16, criterion='entropy')
rf_hpo.fit(X_train, y_train)
rf_score = rf_hpo.score(X_test, y_test)
y_predict = rf_hpo.predict(X_test)
y_true = y_test
print('Accuracy of RF: ' + str(rf_score))
precision, recall, fscore, none = precision_recall_fscore_support(
    y_true, y_predict, average='weighted')
print('Precision of RF: '+(str(precision)))
print('Recall of RF: '+(str(recall)))
print('F1-score of RF: '+(str(fscore)))
print(classification_report(y_true, y_predict))
cm = confusion_matrix(y_true, y_predict)

Accuracy of RF: 0.9814214733203716
Precision of RF: 0.9808432089597965
Recall of RF: 0.9814214733203716
F1-score of RF: 0.9793909056658823
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     18203
           1       0.99      0.99      0.99       381
           2       0.97      1.00      0.99        70
           3       1.00      1.00      1.00        17
           4       1.00      1.00      1.00       926
           5       0.99      1.00      1.00       148
           6       1.00      1.00      1.00       778
           7       0.96      0.96      0.96        56
           8       1.00      1.00      1.00       624
           9       0.56      0.98      0.71       189
          10       1.00      1.00      1.00      1017
          11       0.97      0.45      0.62       261
          12       0.31      0.16      0.22       219
          13       0.75      1.00      0.86         3
          14       1.00      1.00      1.00       

In [70]:
rf_train = rf_hpo.predict(X_train)
rf_test = rf_hpo.predict(X_test)

### Apply ET

In [71]:
et = ExtraTreesClassifier(random_state=0)
et.fit(X_train, y_train)
et_score = et.score(X_test, y_test)
y_predict = et.predict(X_test)
y_true = y_test
print('Accuracy of ET: ' + str(et_score))
precision, recall, fscore, none = precision_recall_fscore_support(
    y_true, y_predict, average='weighted')
print('Precision of ET: '+(str(precision)))
print('Recall of ET: '+(str(recall)))
print('F1-score of ET: '+(str(fscore)))
print(classification_report(y_true, y_predict))
cm = confusion_matrix(y_true, y_predict)

Accuracy of ET: 0.9745085331605098
Precision of ET: 0.9778417795563412
Recall of ET: 0.9745085331605098
F1-score of ET: 0.9748283959632693
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     18203
           1       0.97      0.99      0.98       381
           2       1.00      1.00      1.00        70
           3       0.94      1.00      0.97        17
           4       0.99      1.00      1.00       926
           5       0.99      1.00      1.00       148
           6       0.99      0.99      0.99       778
           7       0.98      1.00      0.99        56
           8       1.00      1.00      1.00       624
           9       0.57      0.96      0.71       189
          10       1.00      1.00      1.00      1017
          11       0.95      0.47      0.63       261
          12       0.14      0.16      0.15       219
          13       1.00      1.00      1.00         3
          14       1.00      1.00      1.00       

#### Hyperparameter optimization (HPO) of extra trees using Bayesian optimization with tree-based Parzen estimator (BO-TPE)

In [76]:
# Hyperparameter optimization of extra trees
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

# Define the objective function
def objective(params):
    params = {
        'n_estimators': int(params['n_estimators']),
        'max_depth': int(params['max_depth']),
        'max_features': int(params['max_features']),
        "min_samples_split": int(params['min_samples_split']),
        "min_samples_leaf": int(params['min_samples_leaf']),
        "criterion": str(params['criterion'])
    }
    clf = ExtraTreesClassifier(**params)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)

    return {'loss': -score, 'status': STATUS_OK}


# Define the hyperparameter configuration space
space = {
    'n_estimators': hp.quniform('n_estimators', 10, 300, 1),
    'max_depth': hp.quniform('max_depth', 5, 50, 1),
    "max_features": hp.quniform('max_features', 1, 21, 1),
    "min_samples_split": hp.quniform('min_samples_split', 2, 11, 1),
    "min_samples_leaf": hp.quniform('min_samples_leaf', 1, 11, 1),
    "criterion": hp.choice('criterion', ['gini', 'entropy'])
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=22)
print("Extra trees: Hyperopt estimated optimum {}".format(best))

100%|██████████| 22/22 [06:30<00:00, 17.77s/trial, best loss: -0.9812918556923742]
Extra trees: Hyperopt estimated optimum {'criterion': 1, 'max_depth': 25.0, 'max_features': 19.0, 'min_samples_leaf': 2.0, 'min_samples_split': 4.0, 'n_estimators': 67.0}


In [77]:
et_hpo = ExtraTreesClassifier(n_estimators=180, 
                              min_samples_leaf=2,
                              max_depth=25, min_samples_split=4, max_features=19, criterion='entropy')
et_hpo.fit(X_train, y_train)
et_score = et_hpo.score(X_test, y_test)
y_predict = et_hpo.predict(X_test)
y_true = y_test
print('Accuracy of ET: ' + str(et_score))
precision, recall, fscore, none = precision_recall_fscore_support(
    y_true, y_predict, average='weighted')
print('Precision of ET: '+(str(precision)))
print('Recall of ET: '+(str(recall)))
print('F1-score of ET: '+(str(fscore)))
print(classification_report(y_true, y_predict))
cm = confusion_matrix(y_true, y_predict)

Accuracy of ET: 0.9811190321883776
Precision of ET: 0.9811492317447232
Recall of ET: 0.9811190321883776
F1-score of ET: 0.9795598426622655
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     18203
           1       0.98      0.99      0.99       381
           2       1.00      1.00      1.00        70
           3       0.94      1.00      0.97        17
           4       1.00      1.00      1.00       926
           5       0.99      1.00      1.00       148
           6       1.00      1.00      1.00       778
           7       0.93      1.00      0.97        56
           8       1.00      1.00      1.00       624
           9       0.56      0.97      0.71       189
          10       1.00      1.00      1.00      1017
          11       0.96      0.46      0.62       261
          12       0.33      0.21      0.25       219
          13       1.00      1.00      1.00         3
          14       1.00      1.00      1.00       

In [78]:
et_train = et_hpo.predict(X_train)
et_test = et_hpo.predict(X_test)

### Apply XGBoost

In [80]:
xg = xgb.XGBClassifier(n_estimators=10)
xg.fit(X_train, y_train)
xg_score = xg.score(X_test, y_test)
y_predict = xg.predict(X_test)
y_true = y_test
print('Accuracy of XGBoost: ' + str(xg_score))
precision, recall, fscore, none = precision_recall_fscore_support(
    y_true, y_predict, average='weighted')
print('Precision of XGBoost: '+(str(precision)))
print('Recall of XGBoost: '+(str(recall)))
print('F1-score of XGBoost: '+(str(fscore)))
print(classification_report(y_true, y_predict))
cm = confusion_matrix(y_true, y_predict)

[19:37:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy of XGBoost: 0.9800820911643984
Precision of XGBoost: 0.9796395703908144
Recall of XGBoost: 0.9800820911643984
F1-score of XGBoost: 0.9781251996390391
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     18203
           1       0.96      0.99      0.97       381
           2       0.99      1.00      0.99        70
           3       0.94      1.00      0.97        17
           4       1.00      1.00      1.00       926
           5       0.99      1.00      1.00       148
           6       1.00      1.00      1.00       778
           7       0.90      0.96      0.93        56
           8       0.99      0.99      0.99

#### Hyperparameter optimization (HPO) of XGBoost using Bayesian optimization with tree-based Parzen estimator (BO-TPE)

In [95]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold

def objective(params):
    params = {
        'n_estimators': int(params['n_estimators']),
        'max_depth': int(params['max_depth']),
        'learning_rate':  abs(float(params['learning_rate']))
    }
    clf = xgb.XGBClassifier(**params,
                            eval_metric='mlogloss')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = accuracy_score(y_test, y_pred)

    return {'loss': -score, 'status': STATUS_OK}


space = {
    'n_estimators': hp.quniform('n_estimators', 10, 100, 5),
    'max_depth': hp.quniform('max_depth', 4, 100, 1),
    'learning_rate': hp.normal('learning_rate', 0.0001, 0.99),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=22)
print("XGBoost: Hyperopt estimated optimum {}".format(best))


100%|██████████| 22/22 [06:21<00:00, 17.35s/trial, best loss: -0.9800820911643984]
XGBoost: Hyperopt estimated optimum {'learning_rate': 0.1995531459500868, 'max_depth': 13.0, 'n_estimators': 100.0}


In [89]:
xg = xgb.XGBClassifier(learning_rate=0.3019765375509075,
                       n_estimators=45, max_depth=5)
xg.fit(X_train, y_train)
xg_score = xg.score(X_test, y_test)
y_predict = xg.predict(X_test)
y_true = y_test
print('Accuracy of XGBoost: ' + str(xg_score))
precision, recall, fscore, none = precision_recall_fscore_support(
    y_true, y_predict, average='weighted')
print('Precision of XGBoost: '+(str(precision)))
print('Recall of XGBoost: '+(str(recall)))
print('F1-score of XGBoost: '+(str(fscore)))
print(classification_report(y_true, y_predict))
cm = confusion_matrix(y_true, y_predict)


[19:53:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy of XGBoost: 0.9814214733203716
Precision of XGBoost: 0.9808749379360137
Recall of XGBoost: 0.9814214733203716
F1-score of XGBoost: 0.979654146434537
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     18203
           1       0.98      0.99      0.99       381
           2       0.97      1.00      0.99        70
           3       1.00      1.00      1.00        17
           4       1.00      1.00      1.00       926
           5       0.99      1.00      1.00       148
           6       1.00      1.00      1.00       778
           7       0.98      0.96      0.97        56
           8       1.00      1.00      1.00 

In [96]:
xg_train = xg.predict(X_train)
xg_test = xg.predict(X_test)

### Apply stacking

In [97]:
base_predictions_train = pd.DataFrame({
    'DecisionTree': dt_train.ravel(),
    'RandomForest': rf_train.ravel(),
    'ExtraTrees': et_train.ravel(),
    'XgBoost': xg_train.ravel(),
})
base_predictions_train.head(5)

,DecisionTree,RandomForest,ExtraTrees,XgBoost
0,0,0,0,0
1,1,1,1,1
2,0,0,0,0
3,4,4,4,4
4,10,10,10,10


In [98]:
dt_train = dt_train.reshape(-1, 1)
et_train = et_train.reshape(-1, 1)
rf_train = rf_train.reshape(-1, 1)
xg_train = xg_train.reshape(-1, 1)
dt_test = dt_test.reshape(-1, 1)
et_test = et_test.reshape(-1, 1)
rf_test = rf_test.reshape(-1, 1)
xg_test = xg_test.reshape(-1, 1)

In [99]:
dt_train.shape

(83287, 1)

In [100]:
x_train = np.concatenate((dt_train, et_train, rf_train, xg_train), axis=1)
x_test = np.concatenate((dt_test, et_test, rf_test, xg_test), axis=1)

In [101]:
stk = xgb.XGBClassifier().fit(x_train, y_train)
y_predict = stk.predict(x_test)
y_true = y_test
stk_score = accuracy_score(y_true, y_predict)
print('Accuracy of Stacking: ' + str(stk_score))
precision, recall, fscore, none = precision_recall_fscore_support(
    y_true, y_predict, average='weighted')
print('Precision of Stacking: '+(str(precision)))
print('Recall of Stacking: '+(str(recall)))
print('F1-score of Stacking: '+(str(fscore)))
print(classification_report(y_true, y_predict))
cm = confusion_matrix(y_true, y_predict)

[21:01:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy of Stacking: 0.9801252970403975
Precision of Stacking: 0.9806980204558118
Recall of Stacking: 0.9801252970403975
F1-score of Stacking: 0.9790631596446348
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     18203
           1       0.99      0.99      0.99       381
           2       0.99      1.00      0.99        70
           3       1.00      1.00      1.00        17
           4       1.00      1.00      1.00       926
           5       0.99      1.00      1.00       148
           6       1.00      1.00      1.00       778
           7       0.98      0.96      0.97        56
           8       1.00      1.00      

#### Hyperparameter optimization (HPO) of the stacking ensemble model (XGBoost) using Bayesian optimization with tree-based Parzen estimator (BO-TPE)

In [107]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold


def objective(params):
    params = {
        'n_estimators': int(params['n_estimators']),
        'max_depth': int(params['max_depth']),
        'learning_rate':  abs(float(params['learning_rate'])),

    }
    clf = xgb.XGBClassifier(**params,  eval_metric='logloss')
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    score = accuracy_score(y_test, y_pred)

    return {'loss': -score, 'status': STATUS_OK}


space = {
    'n_estimators': hp.quniform('n_estimators', 10, 100, 5),
    'max_depth': hp.quniform('max_depth', 4, 100, 1),
    'learning_rate': hp.normal('learning_rate', 0.01, 0.9),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=22)
print("XGBoost: Hyperopt estimated optimum {}".format(best))

100%|██████████| 22/22 [01:00<00:00,  2.75s/trial, best loss: -0.9801252970403975]
XGBoost: Hyperopt estimated optimum {'learning_rate': 0.5706529343600486, 'max_depth': 98.0, 'n_estimators': 35.0}


In [108]:
xg = xgb.XGBClassifier(learning_rate=0.1947844980138047,
                       n_estimators=55, max_depth=55)
xg.fit(x_train, y_train)
xg_score = xg.score(x_test, y_test)
y_predict = xg.predict(x_test)
y_true = y_test
print('Accuracy of XGBoost: ' + str(xg_score))
precision, recall, fscore, none = precision_recall_fscore_support(
    y_true, y_predict, average='weighted')
print('Precision of XGBoost: '+(str(precision)))
print('Recall of XGBoost: '+(str(recall)))
print('F1-score of XGBoost: '+(str(fscore)))
print(classification_report(y_true, y_predict))
cm = confusion_matrix(y_true, y_predict)


[21:11:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy of XGBoost: 0.9801252970403975
Precision of XGBoost: 0.9807250312192217
Recall of XGBoost: 0.9801252970403975
F1-score of XGBoost: 0.979078627833479
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     18203
           1       0.99      0.99      0.99       381
           2       0.99      1.00      0.99        70
           3       1.00      1.00      1.00        17
           4       1.00      1.00      1.00       926
           5       0.99      1.00      1.00       148
           6       1.00      1.00      1.00       778
           7       0.98      0.96      0.97        56
           8       1.00      1.00      1.00 

### Anomaly-based IDS

In [276]:
df = pd.read_csv(
    './CICIDS_2018/CICIDS2018_cleaned.csv')

#### Feature engineering (IG and FCBF)

#### Feature selection by information gain (IG)

In [280]:
from sklearn.feature_selection import mutual_info_classif
importances = mutual_info_classif(X,y)

In [281]:
# calculate the sum of importance scores
f_list = sorted(
    zip(map(lambda x: round(x, 4), importances), features), reverse=True)
Sum = 0
fs = []
for i in range(0, len(f_list)):
    Sum = Sum + f_list[i][0]
    fs.append(f_list[i][1])

In [282]:
# select the important features from top to bottom until the accumulated importance reaches 90%
f_list2 = sorted(
    zip(map(lambda x: round(x, 4), importances/Sum), features), reverse=True)
Sum2 = 0
fs = []
for i in range(0, len(f_list2)):
    Sum2 = Sum2 + f_list2[i][0]
    fs.append(f_list2[i][1])
    if Sum2 >= 0.9:
        break

In [283]:
X_fs = df_frac[fs].values

In [284]:
X_fs.shape

(4009, 45)

#### Feature selection by Fast Correlation Based Filter (FCBF)

In [285]:
from FCBF_module import FCBF, FCBFK, FCBFiP, get_i
fcbf = FCBFK(k=20)

In [286]:
X_fss = fcbf.fit_transform(X_fs, y)

In [287]:
X_fss.shape

(4009, 20)

####  kernel principal component analysis (KPCA)

In [288]:
from sklearn.decomposition import KernelPCA
kpca = KernelPCA(n_components=10, kernel='rbf')
kpca.fit(X_fss, y)
X_kpca = kpca.transform(X_fss)

### Train-test split after feature selection

In [294]:
X_train, X_test, y_train, y_test = train_test_split(
    X_kpca, y, train_size=0.7, test_size=0.3, random_state=0)

### Solve class-imbalance by SMOTE

In [295]:
pd.Series(y_train).value_counts()

0    2263
1     543
dtype: int64

In [296]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(n_jobs=-1)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [297]:
pd.Series(y_test).value_counts()

0    975
1    228
dtype: int64

### Apply the cluster labeling (CL) k-means method

In [298]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN, MeanShift
from sklearn.cluster import SpectralClustering, AgglomerativeClustering, AffinityPropagation, Birch, MiniBatchKMeans, MeanShift
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.metrics import classification_report
from sklearn import metrics

In [309]:
def CL_kmeans(X_train, X_test, y_train, y_test, n):
    km_cluster = KMeans(n_clusters=n)
    result = km_cluster.fit_predict(X_train)
    result2 = km_cluster.predict(X_test)

    count = 0
    a = np.zeros(n)
    b = np.zeros(n)
    for v in range(0, n):
        for i in range(0, len(y_train)):
            if result[i] == v:
                if y_train[i] == 1:
                    a[v] = a[v]+1
                else:
                    b[v] = b[v]+1
    list1 = []
    list2 = []
    for v in range(0, n):
        if a[v] <= b[v]:
            list1.append(v)
        else:
            list2.append(v)
    for v in range(0, len(y_test)):
        if result2[v] in list1:
            result2[v] = 0
        elif result2[v] in list2:
            result2[v] = 1
        else:
            print("-1")
    print(classification_report(y_test, result2))
    cm = confusion_matrix(y_test, result2)
    acc = metrics.accuracy_score(y_test, result2)
    print(str(acc))
    print(cm)

In [323]:
CL_kmeans(X_train, X_test, y_train, y_test, 135)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       975
           1       0.90      0.93      0.91       228

    accuracy                           0.97      1203
   macro avg       0.94      0.95      0.95      1203
weighted avg       0.97      0.97      0.97      1203

0.9667497921862012
[[952  23]
 [ 17 211]]


### Hyperparameter optimization of CL-k-means
Tune "k"

In [324]:
pip install scikit-optimize

You should consider upgrading via the 'c:\Users\flore\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [325]:
#Hyperparameter optimization by BO-GP
from skopt import gp_minimize
import time
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from sklearn import metrics

space = [Integer(2, 50, name='n_clusters')]


@use_named_args(space)
def objective(**params):
    km_cluster = MiniBatchKMeans(batch_size=100, **params)
    n = params['n_clusters']

    result = km_cluster.fit_predict(X_train)
    result2 = km_cluster.predict(X_test)

    count = 0
    a = np.zeros(n)
    b = np.zeros(n)
    for v in range(0, n):
        for i in range(0, len(y_train)):
            if result[i] == v:
                if y_train[i] == 1:
                    a[v] = a[v]+1
                else:
                    b[v] = b[v]+1
    list1 = []
    list2 = []
    for v in range(0, n):
        if a[v] <= b[v]:
            list1.append(v)
        else:
            list2.append(v)
    for v in range(0, len(y_test)):
        if result2[v] in list1:
            result2[v] = 0
        elif result2[v] in list2:
            result2[v] = 1
        else:
            print("-1")
    cm = metrics.accuracy_score(y_test, result2)
    print(str(n)+" "+str(cm))
    return (1-cm)


t1 = time.time()
res_gp = gp_minimize(objective, space, n_calls=20, random_state=0)
t2 = time.time()
print(t2-t1)
print("Best score=%.4f" % (1-res_gp.fun))
print("""Best parameters: n_clusters=%d""" % (res_gp.x[0]))

30 0.9027431421446384
43 0.9010806317539485
43 0.942643391521197
43 0.9459684123025769
32 0.9301745635910225
20 0.8686616791354946
16 0.8603491271820449
5 0.8595178719866999
15 0.857024106400665
25 0.8703241895261845
50 0.9526184538653366
50 0.9376558603491272
50 0.9068994181213632
50 0.9501246882793017
50 0.8927680798004988
40 0.943474646716542
42 0.9177057356608479
50 0.9625935162094763
50 0.9293433083956775
50 0.913549459684123
14.466262340545654
Best score=0.9626
Best parameters: n_clusters=50


In [327]:
#Hyperparameter optimization by BO-TPE
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.cluster import MiniBatchKMeans
from sklearn import metrics


def objective(params):
    params = {
        'n_clusters': int(params['n_clusters']),
    }
    km_cluster = MiniBatchKMeans(batch_size=100, **params)
    n = params['n_clusters']

    result = km_cluster.fit_predict(X_train)
    result2 = km_cluster.predict(X_test)

    count = 0
    a = np.zeros(n)
    b = np.zeros(n)
    for v in range(0, n):
        for i in range(0, len(y_train)):
            if result[i] == v:
                if y_train[i] == 1:
                    a[v] = a[v]+1
                else:
                    b[v] = b[v]+1
    list1 = []
    list2 = []
    for v in range(0, n):
        if a[v] <= b[v]:
            list1.append(v)
        else:
            list2.append(v)
    for v in range(0, len(y_test)):
        if result2[v] in list1:
            result2[v] = 0
        elif result2[v] in list2:
            result2[v] = 1
        else:
            print("-1")
    score = metrics.accuracy_score(y_test, result2)
    print(str(params['n_clusters'])+" "+str(score))
    return {'loss': 1-score, 'status': STATUS_OK}


space = {
    'n_clusters': hp.quniform('n_clusters', 2, 50, 1),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20)
print("Random Forest: Hyperopt estimated optimum {}".format(best))

46 0.9467996674979219                                 
26 0.8877805486284289                                                            
16 0.8420615128844555                                                            
41 0.9310058187863675                                                            
34 0.9093931837073982                                                            
25 0.8769742310889443                                                            
4 0.6974231088944306                                                             
4 0.6774729842061513                                                             
13 0.8320864505403158                                                            
46 0.943474646716542                                                             
45 0.9260182876142976                                                             
32 0.9384871155444722                                                             
49 0.9376558603491272                    

In [335]:
CL_kmeans(X_train, X_test, y_train, y_test, 50)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       975
           1       0.90      0.90      0.90       228

    accuracy                           0.96      1203
   macro avg       0.94      0.94      0.94      1203
weighted avg       0.96      0.96      0.96      1203

0.9617622610141313
[[952  23]
 [ 23 205]]


### Apply the CL-k-means model with biased classifiers

In [336]:
# needs to work on the entire dataset to generate sufficient training samples for biased classifiers
def Anomaly_IDS(X_train, X_test, y_train, y_test, n, b=100):
    # CL-kmeans
    km_cluster = MiniBatchKMeans(n_clusters=n, batch_size=b)
    result = km_cluster.fit_predict(X_train)
    result2 = km_cluster.predict(X_test)

    count = 0
    a = np.zeros(n)
    b = np.zeros(n)
    for v in range(0, n):
        for i in range(0, len(y_train)):
            if result[i] == v:
                if y_train[i] == 1:
                    a[v] = a[v]+1
                else:
                    b[v] = b[v]+1
    list1 = []
    list2 = []
    for v in range(0, n):
        if a[v] <= b[v]:
            list1.append(v)
        else:
            list2.append(v)
    for v in range(0, len(y_test)):
        if result2[v] in list1:
            result2[v] = 0
        elif result2[v] in list2:
            result2[v] = 1
        else:
            print("-1")
    print(classification_report(y_test, result2))
    cm = confusion_matrix(y_test, result2)
    acc = metrics.accuracy_score(y_test, result2)
    print(str(acc))
    print(cm)

    #Biased classifier construction
    count = 0
    print(len(y))
    a = np.zeros(n)
    b = np.zeros(n)
    FNL = []
    FPL = []
    for v in range(0, n):
        al = []
        bl = []
        for i in range(0, len(y)):
            if result[i] == v:
                if y[i] == 1:  # label 1
                    a[v] = a[v]+1
                    al.append(i)
                else:  # label 0
                    b[v] = b[v]+1
                    bl.append(i)
        if a[v] <= b[v]:
            FNL.extend(al)
        else:
            FPL.extend(bl)

    dffp = df.iloc[FPL, :]
    dffn = df.iloc[FNL, :]
    dfva0 = df[df['Label'] == 0]
    dfva1 = df[df['Label'] == 1]

    dffpp = dfva1.sample(n=None, frac=len(
        FPL)/dfva1.shape[0], replace=False, weights=None, random_state=None, axis=0)
    dffnp = dfva0.sample(n=None, frac=len(
        FNL)/dfva0.shape[0], replace=False, weights=None, random_state=None, axis=0)

    dffp_f = pd.concat([dffp, dffpp])
    dffn_f = pd.concat([dffn, dffnp])

    Xp = dffp_f.drop(['Label'], axis=1)
    yp = dffp_f.iloc[:, -1].values.reshape(-1, 1)
    yp = np.ravel(yp)

    Xn = dffn_f.drop(['Label'], axis=1)
    yn = dffn_f.iloc[:, -1].values.reshape(-1, 1)
    yn = np.ravel(yn)

    rfp = RandomForestClassifier(random_state=0)
    rfp.fit(Xp, yp)
    rfn = RandomForestClassifier(random_state=0)
    rfn.fit(Xn, yn)

    dffnn_f = pd.concat([dffn, dffnp])

    Xnn = dffn_f.drop(['Label'], axis=1)
    ynn = dffn_f.iloc[:, -1].values.reshape(-1, 1)
    ynn = np.ravel(ynn)

    rfnn = RandomForestClassifier(random_state=0)
    rfnn.fit(Xnn, ynn)

    X2p = df2.drop(['Label'], axis=1)
    y2p = df2.iloc[:, -1].values.reshape(-1, 1)
    y2p = np.ravel(y2p)

    result2 = km_cluster.predict(X2p)

    count = 0
    a = np.zeros(n)
    b = np.zeros(n)
    for v in range(0, n):
        for i in range(0, len(y)):
            if result[i] == v:
                if y[i] == 1:
                    a[v] = a[v]+1
                else:
                    b[v] = b[v]+1
    list1 = []
    list2 = []
    l1 = []
    l0 = []
    for v in range(0, n):
        if a[v] <= b[v]:
            list1.append(v)
        else:
            list2.append(v)
    for v in range(0, len(y2p)):
        if result2[v] in list1:
            result2[v] = 0
            l0.append(v)
        elif result2[v] in list2:
            result2[v] = 1
            l1.append(v)
        else:
            print("-1")
    print(classification_report(y2p, result2))
    cm = confusion_matrix(y2p, result2)
    print(cm)

In [341]:
Anomaly_IDS(X_train, X_test, y_train, y_test, 42)

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       975
           1       0.85      0.89      0.87       228

    accuracy                           0.95      1203
   macro avg       0.91      0.93      0.92      1203
weighted avg       0.95      0.95      0.95      1203

0.9492934330839568
[[938  37]
 [ 24 204]]
4009


ValueError: Found array with 0 sample(s) (shape=(0, 76)) while a minimum of 1 is required.

# LSTM

#### Import module

In [342]:
import io
import itertools
import pickle
import shutil

import numpy as np
import sklearn.metrics
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow import keras
import os
from tensorflow.python.ops import array_ops

In [344]:
def focal_loss(prediction_tensor, target_tensor, weights=None, alpha=0.25, gamma=2):
    sigmoid_p = tf.nn.sigmoid(prediction_tensor)
    zeros = array_ops.zeros_like(sigmoid_p, dtype=sigmoid_p.dtype)
    pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)
    neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
    per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * tf.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
                          - (1 - alpha) * (neg_p_sub ** gamma) * tf.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
    return tf.reduce_sum(per_entry_cross_ent)


In [345]:
def plot_confusion_matrix(cm, class_names):
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)


In [348]:
# Normalize the confusion matrix.
cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

In [350]:
def log_confusion_matrix():
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(test_traffic)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(test_classes, test_pred)

In [ ]:
disable_classification = False

In [ ]:
training = training.to_numpy(dtype='float32')[:-237, :]
train_traffic = training[:, :-2]
train_labels = training[:, -1].astype(np.uint8)
train_classes = training[:, -2].astype(np.uint8)
testing = testing.to_numpy(dtype='float32')
test_traffic = testing[:, :-2]
test_labels = testing[:, -1].astype(np.uint8)
test_classes = testing[:, -2].astype(np.uint8)